## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from copy import deepcopy
import joblib
from pathlib import Path
import sys
import warnings
from datetime import datetime, timedelta

In [2]:
current_dir = Path.cwd()
sys.path.append(str(current_dir.parent))

## Loading Dataset

In [3]:
weather_df = pd.read_csv("../data/1990-2023-data.csv")
weather_df = weather_df.iloc[:, 1:]
weather_df.head(1)

,date,city_name,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,et0_fao_evapotranspiration,precipitation_forecast
0,01-01-1990,Chennai,27.35,22.3,24.595833,0.1,1,17.429354,30.960001,51.675465,3.471187,0.0


In [4]:
weather_df = weather_df[['date', 'city_name', 'precipitation_sum']]
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%d-%m-%Y')
weather_df

,date,city_name,precipitation_sum
0,1990-01-01,Chennai,0.1
1,1990-01-02,Chennai,0.1
2,1990-01-03,Chennai,0.0
3,1990-01-04,Chennai,0.0
4,1990-01-05,Chennai,8.5
...,...,...,...
211101,2023-12-27,Kakinada,0.0
211102,2023-12-28,Kakinada,0.0
211103,2023-12-29,Kakinada,0.0
211104,2023-12-30,Kakinada,0.0


In [5]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211106 entries, 0 to 211105
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   date               211106 non-null  datetime64[ns]
 1   city_name          211106 non-null  object        
 2   precipitation_sum  211106 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 4.8+ MB


In [6]:
city_names = weather_df['city_name'].unique()
city_names

array(['Chennai', 'Mayiladuthurai', 'Thoothukudi', 'Nagercoil',
       'Thiruvananthapuram', 'Kollam', 'Kochi', 'Kozhikode', 'Kannur',
       'Visakhapatnam', 'Nellore', 'Mangaluru', 'Udupi', 'Mumbai',
       'Daman', 'Alappuzha', 'Kakinada'], dtype=object)

## Feature Scaling

In [7]:
from utils import *
weather_df = split_dataframe(df=weather_df, city_name='Chennai', scaler=None)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
weather_df['precipitation_sum'] = scaler.fit_transform(weather_df[['precipitation_sum']])
weather_df

,precipitation_sum
date,
1990-01-01,-0.318984
1990-01-02,-0.318984
1990-01-03,-0.330588
1990-01-04,-0.330588
1990-01-05,0.655786
...,...
2023-12-27,-0.098500
2023-12-28,-0.098500
2023-12-29,-0.226149


In [29]:
joblib.dump(scaler, "../models/scaler.pkl")

['../models/minmaxscaler.pkl']

In [7]:
scaler = joblib.load("../models/scaler.pkl")
scaler

StandardScaler()

## Data Preparation

In [9]:
LOOKBACK = 180
BATCH_SIZE = 32
EPOCHS = 100
LR = 0.001

In [10]:
from utils import *
from modular.data_setup import *

In [13]:
def get_train_test_data(X, y, train_size, lookback = LOOKBACK):

    train_size = int(len(X) * train_size)

    X_train = X[:train_size]
    X_test = X[train_size:]
    y_train = y[:train_size]
    y_test = y[train_size:]

    X_train = X_train.reshape((-1, lookback, 1))
    X_test = X_test.reshape((-1, lookback, 1))
    y_train = y_train.reshape((-1, 1))
    y_test = y_test.reshape((-1, 1))

    X_train = torch.tensor(X_train, dtype=torch.float)
    X_test = torch.tensor(X_test, dtype=torch.float)
    y_train = torch.tensor(y_train, dtype=torch.float)
    y_test = torch.tensor(y_test, dtype=torch.float)

    return X_train, X_test, y_train, y_test

In [11]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

## Model Training

#### LSTM using PyTorch

In [17]:
from torch.autograd import Variable
from modular.engine import *
from modular.model_builder import *
from modular.utils import *
from modular.data_setup import *

In [18]:
class LSTMModelV0(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers,  output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

        self.relu = nn.ReLU()

        self.fc_1 = nn.Linear(in_features=hidden_size, out_features=hidden_size)

        self.fc = nn.Linear(in_features=hidden_size, out_features=output_size)

    def forward(self, x):
        batch_size = x.size(0)

        h0 = Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))
        c0 = Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))

        out, _ = self.lstm(x, (h0, c0))
        #print("LSTM layer:", out.shape) -> [32, 60 ,8]

        out = self.fc_1(out[:, -1, :])
        #print("Linear Layer 1 :", out.shape) -> [32, 8]

        out = self.relu(out)

        out = self.fc(out)
        #print("linear layer 2:", out.shape)-> [32, 1]

        return self.relu(out)

In [19]:
def set_loss_optimizer(model: nn.Module) -> nn.Module:

    loss_fn = nn.L1Loss()

    optimizer = torch.optim.Adam(model.parameters(), lr = LR)

    return loss_fn, optimizer

In [ ]:
for city in weather_df['city_name'].unique():

    print(f"Starting for city: {city} \n")
    city_weather_df = split_dataframe(df=weather_df, city_name=city)

    city_lag_df = create_lag_dataframe(df=city_weather_df, scaler=scaler, n_steps=LOOKBACK)

    X, y = create_X_y(df=city_lag_df)

    X_train, X_test, y_train, y_test = get_train_test_data(X=X, y=y, train_size=0.75)
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    print(f"Dataset created")

    train_dataloader, test_dataloader = create_dataloader(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
                                                          batch_size=BATCH_SIZE)
    print(f"DataLoader created")

    model = LSTMModelV0(input_size=1, hidden_size=8, num_layers=2, output_size=1)
    print(f"Model Initialized")

    loss_fn, optimizer = set_loss_optimizer(model=model)
    print(f"loss function and optimizer set")

    print(f"Starting training model for city: {city} \n")
    results = train_model(model=model, train_dataloader=train_dataloader, test_dataloader=test_dataloader, loss_fn=loss_fn,
                          optimizer=optimizer, epochs=EPOCHS, device=device)

    print(f"Model trained for city: {city} \n")

    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test.to(device)).to('cpu').numpy()

    save_model(model=model, city=city)

    print("---------------------------------------------------------------\n")
    print("---------------------------------------------------------------\n")

#### LSTM using Tensorflow

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

In [15]:
sequence_length = 180

In [16]:
def LSTMModelV2(input_size, hidden_size, output_size):
    model = Sequential([
        LSTM(100, activation='tanh', input_shape=(input_size, 1)),
        Dense(hidden_size, activation='relu'),
        Dense(output_size, activation='linear')
    ])

    loss_fn = tf.keras.losses.MeanSquaredError()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

    model.compile(optimizer=optimizer, loss=loss_fn)

    return model

In [26]:
for city in weather_df['city_name'].unique():

    print(f"Starting for city: {city} \n")
    city_weather_df = split_dataframe(df=weather_df, city_name=city, scaler=scaler)

    X, y = create_sequence(df=city_weather_df, sequence_length=sequence_length)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Dataset created")

    model = LSTMModelV2(input_size=sequence_length, hidden_size=8, output_size=1)
    print(f"Model Initialized")

    print(f"Starting training model for city: {city} \n")
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1)
    print(f"Model trained for city: {city} \n")

    y_pred = model.predict(X_test)
    y_pred = scaler.inverse_transform(y_pred)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
    print(f"MSE for {city}: {mean_squared_error(y_test, y_pred)}")

    model.save(f'../models/{city}_model.h5')

    print("---------------------------------------------------------------\n")
    print("---------------------------------------------------------------\n")

Starting for city: Chennai 

Dataset created
Model Initialized
Starting training model for city: Chennai 

Epoch 1/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 54ms/step - loss: 0.4218 - val_loss: 0.2126
Epoch 2/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - loss: 0.3894 - val_loss: 0.2048
Epoch 3/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - loss: 0.4080 - val_loss: 0.2177
Epoch 4/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 15s 53ms/step - loss: 0.2911 - val_loss: 0.2576
Epoch 5/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 15s 53ms/step - loss: 0.3550 - val_loss: 0.2205
Model trained for city: Chennai 

77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step


MSE for Chennai: 61.396939825326065
---------------------------------------------------------------

---------------------------------------------------------------

Starting for city: Mayiladuthurai 

Dataset created
Model Initialized
Starting training model for city: Mayiladuthurai 

Epoch 1/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 54ms/step - loss: 0.3499 - val_loss: 0.3329
Epoch 2/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 59ms/step - loss: 0.3396 - val_loss: 0.3090
Epoch 3/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 57ms/step - loss: 0.3166 - val_loss: 0.3089
Epoch 4/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 15s 54ms/step - loss: 0.3245 - val_loss: 0.2985
Epoch 5/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 14s 51ms/step - loss: 0.2953 - val_loss: 0.3008
Model trained for city: Mayiladuthurai 

77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step


MSE for Mayiladuthurai: 45.84452949236239
---------------------------------------------------------------

---------------------------------------------------------------

Starting for city: Thoothukudi 

Dataset created
Model Initialized
Starting training model for city: Thoothukudi 

Epoch 1/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 15s 51ms/step - loss: 0.3791 - val_loss: 0.1402
Epoch 2/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 14s 51ms/step - loss: 0.4353 - val_loss: 0.1062
Epoch 3/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - loss: 0.3460 - val_loss: 0.1210
Epoch 4/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 14s 52ms/step - loss: 0.6298 - val_loss: 0.1255
Epoch 5/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 15s 56ms/step - loss: 0.2811 - val_loss: 0.1320
Model trained for city: Thoothukudi 

77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step


MSE for Thoothukudi: 41.144447895139415
---------------------------------------------------------------

---------------------------------------------------------------

Starting for city: Nagercoil 

Dataset created
Model Initialized
Starting training model for city: Nagercoil 

Epoch 1/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 19s 63ms/step - loss: 0.2918 - val_loss: 0.1547
Epoch 2/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 17s 60ms/step - loss: 0.4047 - val_loss: 0.1534
Epoch 3/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 58ms/step - loss: 0.2516 - val_loss: 0.1580
Epoch 4/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 17s 60ms/step - loss: 0.2613 - val_loss: 0.1743
Epoch 5/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 58ms/step - loss: 0.3021 - val_loss: 0.1564
Model trained for city: Nagercoil 

77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step


MSE for Nagercoil: 45.81542670214307
---------------------------------------------------------------

---------------------------------------------------------------

Starting for city: Thiruvananthapuram 

Dataset created
Model Initialized
Starting training model for city: Thiruvananthapuram 

Epoch 1/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 53ms/step - loss: 0.3638 - val_loss: 0.2716
Epoch 2/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 15s 53ms/step - loss: 0.3041 - val_loss: 0.2726
Epoch 3/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 15s 53ms/step - loss: 0.2748 - val_loss: 0.2697
Epoch 4/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 15s 53ms/step - loss: 0.2536 - val_loss: 0.2737
Epoch 5/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 15s 55ms/step - loss: 0.3393 - val_loss: 0.2857
Model trained for city: Thiruvananthapuram 

77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step


...

MSE for Udupi: 95.0481613397193
---------------------------------------------------------------

---------------------------------------------------------------

Starting for city: Mumbai 

Dataset created
Model Initialized
Starting training model for city: Mumbai 

Epoch 1/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 21s 70ms/step - loss: 0.6024 - val_loss: 0.3836
Epoch 2/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 22s 81ms/step - loss: 0.4396 - val_loss: 0.3838
Epoch 3/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 20s 73ms/step - loss: 0.6386 - val_loss: 0.3909
Epoch 4/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 19s 68ms/step - loss: 0.4836 - val_loss: 0.4038
Epoch 5/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 19s 67ms/step - loss: 0.7210 - val_loss: 0.4382
Model trained for city: Mumbai 

77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step


MSE for Mumbai: 86.91059132609318
---------------------------------------------------------------

---------------------------------------------------------------

Starting for city: Daman 

Dataset created
Model Initialized
Starting training model for city: Daman 

Epoch 1/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 22s 70ms/step - loss: 0.5045 - val_loss: 0.3161
Epoch 2/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 19s 70ms/step - loss: 0.4758 - val_loss: 0.3119
Epoch 3/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 19s 67ms/step - loss: 0.4354 - val_loss: 0.3319
Epoch 4/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 19s 69ms/step - loss: 0.4451 - val_loss: 0.3061
Epoch 5/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 17s 62ms/step - loss: 0.4892 - val_loss: 0.3719
Model trained for city: Daman 

77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step


MSE for Daman: 157.64084499328848
---------------------------------------------------------------

---------------------------------------------------------------

Starting for city: Alappuzha 

Dataset created
Model Initialized
Starting training model for city: Alappuzha 

Epoch 1/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 21s 68ms/step - loss: 0.4658 - val_loss: 0.3052
Epoch 2/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 59ms/step - loss: 0.3983 - val_loss: 0.3136
Epoch 3/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 57ms/step - loss: 0.4113 - val_loss: 0.3237
Epoch 4/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 57ms/step - loss: 0.3355 - val_loss: 0.3050
Epoch 5/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 56ms/step - loss: 0.3958 - val_loss: 0.3077
Model trained for city: Alappuzha 

77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step


MSE for Alappuzha: 46.45887610019766
---------------------------------------------------------------

---------------------------------------------------------------

Starting for city: Kakinada 

Dataset created
Model Initialized
Starting training model for city: Kakinada 

Epoch 1/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step - loss: 0.3444 - val_loss: 0.5153
Epoch 2/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 57ms/step - loss: 0.4768 - val_loss: 0.4100
Epoch 3/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 56ms/step - loss: 0.7008 - val_loss: 0.3681
Epoch 4/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 58ms/step - loss: 0.3435 - val_loss: 0.3579
Epoch 5/5
276/276 ━━━━━━━━━━━━━━━━━━━━ 16s 56ms/step - loss: 0.3129 - val_loss: 0.3589
Model trained for city: Kakinada 

77/77 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step


MSE for Kakinada: 72.67869352594961
---------------------------------------------------------------

---------------------------------------------------------------



In [27]:
model = tf.keras.models.load_model("models/Chennai_model.h5")
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_11 (LSTM)                  │ (None, 100)            │        40,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 8)              │           808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,619 (162.58 KB)

 Trainable params: 41,617 (162.57 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)